<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Benjamin Torrealba 
- Nombre de alumno 2: Israel Rodríguez


### **Link de repositorio de GitHub:** https://github.com/BnjmnNicholas/MDS7202-2023-2

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [2]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [3]:
def custom_features(dataframe_in):
    pass
    

In [3]:
def feature_length(dataframe_in):
    # Primero se calcula la fecha de la primera visita de cada cliente
    first_visit = dataframe_in.groupby('Customer ID')['InvoiceDate'].min().reset_index()
    first_visit.columns = ['Customer ID', 'FirstVisit']

    # Obtener la última visita de cada customer
    last_visit = dataframe_in.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_visit.columns = ['Customer ID', 'LastVisit']

    # Calcula el Length de cada customer
    length = last_visit['LastVisit'] - first_visit['FirstVisit']
    length = length.dt.days

    # Se crea el dataframe de salida
    length = pd.DataFrame({'Customer ID': last_visit['Customer ID'], 'Length': length})
    return length

def feature_recency(dataframe_in):
    # Obtiene la última visita de cada customer
    last_visit = dataframe_in.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_visit.columns = ['Customer ID', 'LastVisit']
    
    # Obtiene la fecha más reciente de la base de datos y le suma un dia
    max_date = dataframe_in['InvoiceDate'].max() + datetime.timedelta(days=1)

    # Calcula la recencia
    last_visit['Recency'] = (max_date - last_visit['LastVisit']).dt.days
    last_visit = last_visit.drop(columns=['LastVisit'])

    return last_visit

def feature_monetary(dataframe_in):
    # obtiene la cantidad total de dinero gastado por cada visita, esto multiplicando la cantidad por el precio
    totalprice = dataframe_in['Quantity'] * dataframe_in['Price']
    totalprice = pd.DataFrame({'totalprice': totalprice})
    totalprice = pd.concat([dataframe_in['Customer ID'], dataframe_in['InvoiceDate'], totalprice], axis=1)

    # obtiene la cantidad de fechas distintas asociadas a un customer id
    count = totalprice.groupby(['Customer ID']).count().reset_index().drop(columns=['totalprice'])

    # suma los valores de totalprice si el invoiceDAte es el mismo
    monetary = totalprice.groupby(['Customer ID']).sum().reset_index()

    # divide el totalprice por la cantidad de fechas distintas
    monetary['totalprice'] = monetary['totalprice'] / count['InvoiceDate']
    monetary = monetary.rename(columns={'totalprice': 'Monetary'})

    return monetary

def feature_frecuency(dataframe_in):
    # hace una copia del dataframe con solo las columnas customer id y invoice date
    frequency = dataframe_in[['Customer ID', 'InvoiceDate']].copy()

    # transforma la columna invoice date a datetime de tipo dia/mes/año y hora
    frequency['InvoiceDate'] = pd.to_datetime(frequency['InvoiceDate'], format='%d/%m/%Y %H:%M').dt.strftime('%d/%m/%Y %H')

    # renombramos la columna invoice date a frequency
    frequency = frequency.rename(columns={'InvoiceDate': 'Frequency'})
    
    # obtiene la cantidad de fechas distintas asociadas a un customer id y las agrega como una columna frequency
    frequency = frequency.groupby(['Customer ID']).nunique().reset_index()

    return frequency


def feature_periodicity(dataframe_in):
    # hace una copia del dataframe con solo las columnas customer id y invoice date
    periodicity = dataframe_in[['Customer ID', 'InvoiceDate']].copy()

    # transforma la columna invoice date a datetime de tipo dia/mes/año
    periodicity['InvoiceDate'] = pd.to_datetime(periodicity['InvoiceDate']).dt.date
    periodicity = periodicity.drop_duplicates()

    # crea un dataframe que contiene customer id y una columna llamada fechas que contiene una lista con las fechas de visita
    periodicity = periodicity.groupby(['Customer ID'])['InvoiceDate'].apply(list).reset_index()

    # en las listas de fechas, se calcula la diferencia entre cada fecha y se agrega como una nueva columna
    periodicity['Periodicity'] = periodicity['InvoiceDate'].apply(lambda x: np.diff(x))

    # drop de la columna invoice date
    periodicity = periodicity.drop(columns=['InvoiceDate'])

    # pasa todas los valores de las listas a enteros
    periodicity['Periodicity'] = periodicity['Periodicity'].apply(lambda x: [i.days for i in x])

    # calcula la desviación estandar de cada lista
    periodicity['Periodicity'] = periodicity['Periodicity'].apply(lambda x: np.std(x))

    return periodicity


In [4]:
feature_frecuency(df_retail).sample(10)

,Customer ID,Frequency
1764,14813.0,3
102,12492.0,1
2122,15294.0,1
3287,16905.0,10
2003,15135.0,1
3749,17514.0,1
3502,17195.0,1
643,13280.0,3
832,13554.0,6
631,13262.0,2


In [5]:
feature_periodicity(df_retail)

b:\GITPROJECTS\LAB MDS\mdsEnv2\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,Customer ID,Periodicity
0,12346.0,39.814849
1,12347.0,0.000000
2,12348.0,NaN
3,12349.0,72.000000
4,12351.0,NaN
...,...,...
4309,18283.0,53.651654
4310,18284.0,NaN
4311,18285.0,NaN
4312,18286.0,0.000000


**Resúesta de Pauta:**

In [6]:
def custom_feature(dataframe_in):
    """
    Esta funcion se encarga de llamar a todas las funciones que generan las features y las une en un solo dataframe
    
    args:
        dataframe_in: dataframe de entrada
        
    return:
        dataframe con todas las features
    """
    df_feature_length = feature_length(dataframe_in)
    df_feature_recency = feature_recency(dataframe_in)
    df_feature_monetary = feature_monetary(dataframe_in)
    df_feature_frecuency = feature_frecuency(dataframe_in)
    df_feature_periodicity = feature_periodicity(dataframe_in)
    
    
    # merge de las features
    features = df_feature_length.merge(df_feature_recency, on='Customer ID')
    features = features.merge(df_feature_frecuency, on='Customer ID')
    features = features.merge(df_feature_monetary, on='Customer ID')
    features = features.merge(df_feature_periodicity, on='Customer ID')
    
    # Se reemplazan los valores NaN por 0
    features = features.fillna(0)
    return features
    
    

In [7]:
df_out = custom_feature(df_retail)

C:\Users\Benjamin Nicholas\AppData\Local\Temp\ipykernel_34536\3072890824.py:42: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monetary = totalprice.groupby(['Customer ID']).sum().reset_index()
b:\GITPROJECTS\LAB MDS\mdsEnv2\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [8]:
df_out

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,165,8,11.298788,39.814849
1,12347.0,37,3,2,18.638310,0.000000
2,12348.0,0,74,1,11.108000,0.000000
3,12349.0,181,43,3,26.187647,72.000000
4,12351.0,0,11,1,14.330000,0.000000
...,...,...,...,...,...,...
4309,18283.0,275,18,5,2.854240,53.651654
4310,18284.0,0,67,1,16.488571,0.000000
4311,18285.0,0,296,1,35.583333,0.000000
4312,18286.0,247,112,2,19.349701,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [15]:
from sklearn.preprocessing import MinMaxScaler

class MinMax(BaseEstimator, TransformerMixin):
    """
    Esta clase se encarga de normalizar todas las columnas del dataframe usando ColumnTransformer()
    """
    def __init__(self):
        self.columnas = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # se obtienen las columnas del dataframe sin contar la columna customer id, se almacena en una lista
        self.columnas = X.columns[1:]

        # Definir el preprocesamiento utilizando ColumnTransformer
        preprocessor = ColumnTransformer([('num', MinMaxScaler(), self.columnas)])

        # Aplicar el preprocesamiento al DataFrame X
        X = preprocessor.fit_transform(X)
        X = pd.DataFrame(X, columns=self.columnas)
    
        return X
    
    def inverse_transform(self, X):
        # se obtienen las columnas del dataframe sin contar la columna customer id, se almacena en una lista
        self.columnas = X.columns[1:]

        # Definir el preprocesamiento utilizando ColumnTransformer
        preprocessor = ColumnTransformer([('num', MinMaxScaler(), self.columnas)])

        # Aplicar el preprocesamiento al DataFrame X
        X = preprocessor.inverse_transform(X)
        X = pd.DataFrame(X, columns=self.columnas)
    
        return X


In [16]:
minmax_instance = MinMax()
minmax_instance.fit(df_out)
minmax_instance.transform(df_out)

,Length,Recency,Frequency,Monetary,Periodicity
0,0.525469,0.439678,0.040230,0.001032,0.219972
1,0.099196,0.005362,0.005747,0.001702,0.000000
2,0.000000,0.195710,0.000000,0.001014,0.000000
3,0.485255,0.112601,0.011494,0.002391,0.397790
4,0.000000,0.026810,0.000000,0.001308,0.000000
...,...,...,...,...,...
4309,0.737265,0.045576,0.022989,0.000261,0.296418
4310,0.000000,0.176944,0.000000,0.001505,0.000000
4311,0.000000,0.790885,0.000000,0.003249,0.000000
4312,0.662198,0.297587,0.005747,0.001767,0.000000


In [21]:
class MinMax(BaseEstimator, TransformerMixin):
    """
    Esta clase se encarga de normalizar todas las columnas del dataframe usando ColumnTransformer()
    """
    def __init__(self):
        self.columnas = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # se obtienen las columnas del dataframe sin contar la columna customer id, se almacena en una lista
        self.columnas = X.columns[1:]
        
        # obtenemos los maximmos y minimos de cada columna
        maximos = X[self.columnas].max()
        minimos = X[self.columnas].min()
        
        # normalizamos cada columna
        X[self.columnas] = (X[self.columnas] - minimos) / (maximos - minimos)
        
    
        return X
    



In [18]:
minmax_instance = MinMax()
minmax_instance.fit(df_out)
minmax_instance.transform(df_out)

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,0.525469,0.439678,0.040230,0.001032,0.219972
1,12347.0,0.099196,0.005362,0.005747,0.001702,0.000000
2,12348.0,0.000000,0.195710,0.000000,0.001014,0.000000
3,12349.0,0.485255,0.112601,0.011494,0.002391,0.397790
4,12351.0,0.000000,0.026810,0.000000,0.001308,0.000000
...,...,...,...,...,...,...
4309,18283.0,0.737265,0.045576,0.022989,0.000261,0.296418
4310,18284.0,0.000000,0.176944,0.000000,0.001505,0.000000
4311,18285.0,0.000000,0.790885,0.000000,0.003249,0.000000
4312,18286.0,0.662198,0.297587,0.005747,0.001767,0.000000


In [23]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.min_vals_ = X.min()
        self.max_vals_ = X.max()
        return self

    def transform(self, X):
        return (X - self.min_vals_) / (self.max_vals_ - self.min_vals_)



min_max_transformer = ColumnTransformer(transformers=[
    ('min_max', MinMax(), df_out.columns[1:])
])

transformed_data = min_max_transformer.fit_transform(df_out)
transformed_df = pd.DataFrame(transformed_data, columns=df_out.columns[1:])





In [24]:
transformed_df

,Length,Recency,Frequency,Monetary,Periodicity
0,0.525469,0.439678,0.040230,0.001032,0.219972
1,0.099196,0.005362,0.005747,0.001702,0.000000
2,0.000000,0.195710,0.000000,0.001014,0.000000
3,0.485255,0.112601,0.011494,0.002391,0.397790
4,0.000000,0.026810,0.000000,0.001308,0.000000
...,...,...,...,...,...
4309,0.737265,0.045576,0.022989,0.000261,0.296418
4310,0.000000,0.176944,0.000000,0.001505,0.000000
4311,0.000000,0.790885,0.000000,0.003249,0.000000
4312,0.662198,0.297587,0.005747,0.001767,0.000000


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [56]:
t_sne = Pipeline(steps=[("preprocesamiento",FunctionTransformer(custom_feature)), ("MinMax",MinMax()), ("TSNE",TSNE(n_components=2, random_state=42)) ])

In [57]:
t_sne.fit(df_retail)

C:\Users\hakup\AppData\Local\Temp\ipykernel_24812\3072890824.py:42: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

e:\Python\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice



Pipeline(steps=[('preprocesamiento',
                 FunctionTransformer(func=<function custom_feature at 0x000001FEF0174040>)),
                ('MinMax', MinMax()), ('TSNE', TSNE(random_state=42))])

In [62]:
custom_features = pd.DataFrame(
    t_sne.fit_transform(df_retail)
)

custom_features

C:\Users\hakup\AppData\Local\Temp\ipykernel_24812\3072890824.py:42: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

e:\Python\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice



,0,1
0,-5.223009,16.786797
1,-2.655370,-37.800041
2,-64.778336,-15.635365
3,15.530519,23.240274
4,17.787386,-47.388725
...,...,...
4309,27.369024,15.589514
4310,-59.538212,-35.550808
4311,-5.672531,44.750023
4312,0.845239,6.336861


In [64]:
# Primera componente principal respecto al indice
fig = px.scatter(custom_features, y=0, x=custom_features.index)
fig.show()


In [63]:
# Segunda componente
fig = px.scatter(custom_features, y=1, x=custom_features.index)
fig.show()

In [65]:
# Primera componente versus Segunda componente
fig = px.scatter(custom_features, y=1, x=0)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [67]:
intertias = [
    [i, 
     Pipeline(steps=[
         ("preprocesamiento",FunctionTransformer(custom_feature)), 
         ("MinMax",MinMax()), 
         ("Kmeans",KMeans(n_clusters=i, random_state=99))]).fit(df_retail).named_steps["Kmeans"].inertia_]
    for i in range(1, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

C:\Users\hakup\AppData\Local\Temp\ipykernel_24812\3072890824.py:42: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

e:\Python\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice

e:\Python\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\hakup\AppData\Local\Temp\ipykernel_24812\3072890824.py:42: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

e:\Python\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning:

D

,n° clusters,inertia
0,1,926.651386
1,2,382.332250
2,3,217.962888
3,4,157.886093
4,5,132.975443
5,6,111.002508
6,7,96.281297
7,8,82.695067
8,9,75.542172
9,10,69.313880


In [68]:
# se grafican los valores de inertia según la cantidad de clusters
fig = px.line(intertias, y="inertia", x="n° clusters")
fig.show()

##### argumentación del 8

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [82]:
k_means = Pipeline(steps=[("preprocesamiento",FunctionTransformer(custom_feature)), ("MinMax",MinMax()), ("Kmeans",KMeans(n_clusters=7, random_state=99))])

k_means.fit(df_retail)

# Se obtienen los centroides de cada cluster y se almacenan en un dataframe
centroids = pd.DataFrame(k_means.named_steps["Kmeans"].cluster_centers_, columns=df_out.columns[1:])

# Se invierte la normalización de los centroides
centroids = k_means.named_steps["MinMax"].inverse_transform(centroids)
centroids

C:\Users\hakup\AppData\Local\Temp\ipykernel_24812\3072890824.py:42: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

e:\Python\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice

e:\Python\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



AttributeError: 'ColumnTransformer' object has no attribute 'inverse_transform'

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [78]:
# se obtiene un dataframe con los puntos de cada cluster
clusters = pd.DataFrame(k_means.named_steps["Kmeans"].labels_, columns=["cluster"])

# Primera componente versus Segunda componente
fig = px.scatter(custom_features, y=1, x=0, color=clusters["cluster"])
fig.show()

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>